In [1]:
import csv
import pandas as pd
from tqdm import tqdm, trange

In [2]:
restaurant_df = pd.read_csv('ichefdata/ichefdata/ichef 資料.csv')
info_df = restaurant_df[['店名', '店址']]
info_df.rename(columns = {'店名':'name', '店址':'address'}, inplace = True)
info_df

/tmp/ipykernel_648336/3052713340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_df.rename(columns = {'店名':'name', '店址':'address'}, inplace = True)


,name,address
0,% shabu,桃園市桃園區中埔ㄧ街331號
1,% shabu,桃園市桃園區中埔ㄧ街331號
2,% shabu,桃園市桃園區中埔ㄧ街331號
3,% shabu,桃園市桃園區中埔ㄧ街331號
4,% shabu,桃園市桃園區中埔ㄧ街331號
...,...,...
188038,ＴＡＣＢ,高雄市新興區林森二路135巷35號
188039,ＴＡＣＢ,高雄市新興區林森二路135巷35號
188040,ＴＡＣＢ,高雄市新興區林森二路135巷35號
188041,ＴＡＣＢ,高雄市新興區林森二路135巷35號


In [3]:
info_df = info_df.drop_duplicates().reset_index(drop=True)
info_df

,name,address
0,% shabu,桃園市桃園區中埔ㄧ街331號
1,(停用）喜樂咖啡,台中市西區美村路一段687號2樓
2,055龍哥斯特,花蓮縣吉安鄉中央路三段458號
3,0870 Kitchen & Bar,台北市中山區中山北路二段137巷45號
4,1+咖啡館,高雄市苓雅區永定街91號
...,...,...
3783,龍波斯特龍江店,台北市中山區龍江路100巷1號1樓
3784,龍門馬場洞韓牛燒肉,台中市北屯區軍功路二段185號
3785,?壹號。拌,台北市信義區永吉路340號1樓
3786,Ｍr.Health 舒肥廚房,臺中市龍井區台灣大到五段三巷42弄22號


In [4]:
print(len(info_df['name'].unique()))
print(len(info_df['address'].unique()))

3778
3701


In [5]:
info = info_df.groupby('address')['name'].apply(list).reset_index(name='name')
info

,address,name
0,0,[NDBC-VN]
1,100台灣台北市中正區濟南路二段48巷3-2號,[禮面作]
2,103台灣台北市大同區延平北路一段69巷5號2F,[Retro印刷]
3,104台北市中山區一江街25號1樓,[parc.c.studio]
4,104台北市中山區雙城街13巷18號2樓 (平信 18:00開店),[Ansleep]
...,...,...
3695,高雄市鼓山區青海路277號,[BoBo Rose]
3696,高雄市鼓山區鼓元街57號一樓,[My Cofi 哈瑪星]
3697,高雄市鼓山區鼓南街30號1樓,[木葉粗食]
3698,高雄市鼓山區鼓山一路53巷107號,[光悅茶屋]


In [6]:
for i in range(info.shape[0]):
    if len(info.iloc[i]['name']) > 1:
            print(info.iloc[i])

address    南投縣草屯鎮墩煌路三段168號
name          [督賀咖啡, 督賀蔬苑]
Name: 72, dtype: object
address    台中市北屯區昌平路一段105-12號
name            [威霸鍋物, 米可烘焙坊]
Name: 180, dtype: object
address     台中市北屯區熱河路二段128號
name       [憲賣咖啡, 憲賣咖啡(外帶)]
Name: 194, dtype: object
address                    台中市后里區月眉北路486號
name       [星月大地 咖啡廳, 星月大地 售票處, 星月大地 宴會館]
Name: 270, dtype: object
address                   台中市太平區中山路四段170號
name       [漢家園 中山店 第一結帳口, 漢家園 中山店 第二結帳口]
Name: 293, dtype: object
address       台中市太平區旱溪東路二段330號
name       [威霸米可烘培坊, 威霸鍋物(旱溪)]
Name: 297, dtype: object
address               台中市梧棲區台灣大道十段168號
name       [共?Gunglok, 冰見海鮮丼?鮨, 金?製麵處]
Name: 316, dtype: object
address              台中市西區存中街46號
name       [Go?t Bar 好吧, Gubami]
Name: 418, dtype: object
address               台中市西區民生路368巷4弄14號
name       [審計新村-甜月亮, 審計新村-甜月亮手工義大利冰淇淋]
Name: 451, dtype: object
address       台中市西屯區台灣大道四段1727號
name       [東海二餐A, 東海管餐, 東海花餐A]
Name: 499, dtype: object
address      台中市西屯區國安一路168號1樓
name       [宇良食健康鍋物, 赤沐TAIKO]
Nam

In [8]:
from crawler import WebDriver
import time
import random
import json
from json.decoder import JSONDecodeError

web_source = 'https://www.google.com/search?q=mcdonald&sca_esv=556221820&tbm=lcl&ei=iSjXZMX7HMuQhwPnqJbgAg&oq=mcd&gs_lp=Eg1nd3Mtd2l6LWxvY2FsIgNtY2QqAggAMg0QABiKBRixAxiDARhDMgcQABiKBRhDMgcQABiKBRhDMgcQABiKBRhDMgcQABiKBRhDMgcQABiKBRhDMgUQABiABDIFEAAYgAQyBRAAGIAEMggQABiABBixA0jZQVAAWPg5cA94AJABAJgBKaABtgKqAQE5uAEDyAEA-AEBqAIAwgILEAAYgAQYsQMYgwHCAgsQABiKBRixAxiDAcICChAAGIoFGLEDGEOIBgE&sclient=gws-wiz-local#rlfi=hd:;si:;mv:[[25.0579939,121.5714136],[24.995991099999998,121.51168539999999]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:4'
driver = WebDriver()
driver.get_url(web_source)
driver.implicitly_wait(20)

search_result = []

with open('restaurant.json', 'r', encoding='utf-8') as infile:
    try:
        json_data = json.load(infile)
        print('previous : ', json_data.keys())
    except JSONDecodeError:
        json_data = {}

with open('restaurant.json', 'w', encoding='utf-8') as outfile:
    for i in trange(info_df.shape[0], position=0):
        try:
            # json_data = json.load(infile)
            address, name = info_df.iloc[i]['address'], info_df.iloc[i]['name']
            
            # json_data = json.load(open('restaurant.json', encoding='utf-8'))

            if name in json_data:
                continue

            result = driver.search_restaurant(name, address)

            if result == (None, None):
                ratings = None
            else:
                driver.implicitly_wait(20)
                try:
                    _, ratings = driver.get_reviews()
                    if ratings:
                        for timestamp in ratings.keys():
                            ratings[timestamp] = sum(ratings[timestamp]) / len(ratings[timestamp])

                        ratings = dict(ratings)
                except:
                    ratings = None

            json_data[name] = ((name, address), result, ratings)

            # print(json_data[name])

            outfile.seek(0)
            json.dump(json_data, outfile, ensure_ascii = False)

            time.sleep(random.randint(1, 2))
            driver.driver.delete_all_cookies()
        except:
            # driver.driver.delete_all_cookies()
            # uselessWindows = driver.driver.window_handles
            # driver.driver.switch_to.window(uselessWindows[-1]) 
            # driver.driver.close()

            driver = WebDriver()
            driver.get_url(web_source)
            driver.implicitly_wait(20)
            i -= 1
            print('new driver')

previous :  dict_keys(['% shabu', '(停用）喜樂咖啡', '055龍哥斯特', '0870 Kitchen & Bar', '1+咖啡館', '1001手作廚房', '101小飯廳', '108抹茶茶廊 Sogo復興店', '108抹茶茶廊 信義誠品店', '11HomeKitchen', '11pasta', '12 Cafe', '13小館', '152番地', '16廚房素食餐廳', '18號餐酒館', '1942美式殺殺鍋', '1942餓棧餐廳', '1975 Antique’s Cafr Tea Room', '1980義大利麵', '1982 de glac?e 法式冰淇淋', '1989K cafe', '19號水岸咖啡館', '2/F義大利麵', '23 PUBLIC', '23 Public Beitou 北投', '247火山咖啡', '28Bakery&coffee', '2S', '2gether', '392cafe', '3DPW祥貿科技企業有限公司', '3Pig韓式燒肉', '44號館手工烘焙咖啡', '45號咖啡', '4F小飯館-淡江店', '50,45 wine', '50,50 Cuisine Fran?aise', '555薑母鴨', '571 street', '58咖啡行館', '5味臭臭鍋 龜山德明店', '612號月光海洋咖啡館', '62早餐屋', '69奇點廚房', '6號咖啡館', '7.2cafe', '70 salon', '8292', '82mm Cafe', '8528 music bar', '87霸氣烤魚火鍋', '88kbbq', '8more 台灣第一家白木耳專賣店', '8more白木耳專賣店 校前店', '8荳輕食', '900度串燒工坊', '923手作甜點', '9527 Sports & Music Bar', '953', 'A Table', 'A&G La Fusione義式餐廳', 'A-bao豐南', 'A.Bistro', 'A1車飾', 'AF火鍋', 'AIYO cafe', 'ALCO BAR', 'ALL DAY ROASTING COMPANY', 'ALLEZ BISTRO', 'AM patisserie et priva

 85%|████████▌ | 3228/3788 [1:15:27<63:41:09, 409.41s/it]

new driver


 86%|████████▌ | 3244/3788 [2:10:22<57:29:44, 380.49s/it]

new driver


...

 99%|█████████▊| 3734/3788 [19:49:31<5:00:13, 333.58s/it] 

new driver


 99%|█████████▊| 3735/3788 [20:04:24<7:22:52, 501.38s/it]

new driver


100%|██████████| 3788/3788 [23:51:02<00:00, 22.67s/it]     


In [ ]:
# for i in range(info_df.shape[0]):
#     address, name = info_df.iloc[i]['address'], info_df.iloc[i]['name']
#     driver.search_restaurant(name, address)
#     driver.implicitly_wait(20)
#     reviews, ratings = driver.get_reviews()

#     for time in ratings.keys():
#         ratings[time] = sum(ratings[time]) / len(ratings[time])

#     print(dict(ratings))